In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

!pip install beautifulsoup4 lxml
from bs4 import BeautifulSoup

%matplotlib inline

# Fetching data

In [2]:
# Crawl data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get(url)
soup = BeautifulSoup(res.text)

Parse data by using `beautifulsoup` and `pandas`. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

In [26]:
df_raw = pd.read_html(str(soup.select_one('table.wikitable')))[0]
df_raw

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [28]:
df_raw.Borough.unique()

array(['Not assigned', 'North York', 'Downtown Toronto', 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [29]:
# Drop row with Borough == 'Not assigned'
df = df_raw[df_raw.Borough != 'Not assigned']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [50]:
df[df.Neighbourhood == 'Not assigned']['Not assigned'] = df[df.Neighbourhood == 'Not assigned'].Borough
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M6A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [52]:
df_final = df.groupby('Postcode', as_index=False).agg({
    'Borough': 'first',
    'Neighbourhood': ', '.join
}).sort_values(by='Postcode')

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [53]:
df_final.shape

(103, 3)

In [54]:
df_final.head(101)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
